In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

In [2]:
!gdown https://drive.google.com/uc?id=1VKuCi2vNFZLhmyy2CdU0Lp8-Ab3re81N 
!gdown https://drive.google.com/uc?id=1JaF3TKLcveF2-88mzqF9EgP8928JhpCB
!gdown https://drive.google.com/uc?id=13ORjoqdKbALvDyVjPIq23HWleQhccafJ
!gdown https://drive.google.com/uc?id=1oijYQho6s63CjHppuJT7ERaRb81Javkt

Downloading...
From: https://drive.google.com/uc?id=1VKuCi2vNFZLhmyy2CdU0Lp8-Ab3re81N
To: /content/interactions_test.csv
100% 523k/523k [00:00<00:00, 111MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JaF3TKLcveF2-88mzqF9EgP8928JhpCB
To: /content/interactions_train.csv.zip
100% 9.29M/9.29M [00:00<00:00, 105MB/s]
Downloading...
From: https://drive.google.com/uc?id=13ORjoqdKbALvDyVjPIq23HWleQhccafJ
To: /content/RAW_interactions.csv.zip
100% 131M/131M [00:01<00:00, 93.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oijYQho6s63CjHppuJT7ERaRb81Javkt
To: /content/RAW_recipes.csv.zip
100% 87.3M/87.3M [00:00<00:00, 143MB/s]


In [3]:
! echo y | unzip RAW_interactions.csv.zip 
! echo y | unzip RAW_recipes.csv.zip
! echo y | unzip interactions_train.csv.zip

Archive:  RAW_interactions.csv.zip
replace RAW_interactions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: RAW_interactions.csv    
Archive:  RAW_recipes.csv.zip
replace RAW_recipes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: RAW_recipes.csv         
Archive:  interactions_train.csv.zip
replace interactions_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: interactions_train.csv  


In [82]:
comments = pd.read_csv('RAW_interactions.csv')
recipes = pd.read_csv('RAW_recipes.csv')

In [83]:
comments['sentiment']= comments['rating']>=3

### Remarques:


De nombreuses recettes ont une note de 0 sans porter de jugement négatif ou positif sur les recettes. Essayons de "sauver" certaines données en récupérant les recette ayant une note nulle mais dont la review contient des éléments de vocabulaire positif.

In [84]:
comments[comments['rating']==0]

,user_id,recipe_id,date,rating,review,sentiment
6,124416,120345,2011-08-06,0,"Just an observation, so I will not rate. I fo...",False
12,468945,134728,2008-02-20,0,Made my own buttermilk w/ vinegar and milk. U...,False
23,681408,225241,2008-03-14,0,This is a very good recipe. We also want to c...,False
39,2002068522,29093,2018-03-26,0,i made it and it was amazing,False
56,980899,79222,2010-10-18,0,"I used half cream and half 1% milk, added anot...",False
...,...,...,...,...,...,...
1132318,1197076,190261,2010-03-08,0,Delicious ! I tweeked the recipe a bit>>substi...,False
1132323,2405600,290157,2014-11-10,0,Just added this mix to a homemade beef &amp; s...,False
1132325,2000650711,290157,2017-04-10,0,Would this make a good hamburger patty seasoning?,False
1132361,102526,54493,2007-04-26,0,I am not going to rate this because I did have...,False


In [85]:
good = ['great','marvelous','delicious','love','awesome','amazing','yummy','unbelievable','fabulous']

In [86]:
comments['review']= comments['review'].str.lower()

In [87]:
mask = (comments['rating'] == 0) & (comments['review'].str.contains('|'.join(good)))

In [88]:
comments['sentiment'][mask]= True

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [89]:
comments[comments['rating']==0]

,user_id,recipe_id,date,rating,review,sentiment
6,124416,120345,2011-08-06,0,"just an observation, so i will not rate. i fo...",True
12,468945,134728,2008-02-20,0,made my own buttermilk w/ vinegar and milk. u...,False
23,681408,225241,2008-03-14,0,this is a very good recipe. we also want to c...,False
39,2002068522,29093,2018-03-26,0,i made it and it was amazing,True
56,980899,79222,2010-10-18,0,"i used half cream and half 1% milk, added anot...",True
...,...,...,...,...,...,...
1132318,1197076,190261,2010-03-08,0,delicious ! i tweeked the recipe a bit>>substi...,True
1132323,2405600,290157,2014-11-10,0,just added this mix to a homemade beef &amp; s...,False
1132325,2000650711,290157,2017-04-10,0,would this make a good hamburger patty seasoning?,False
1132361,102526,54493,2007-04-26,0,i am not going to rate this because i did have...,False


In [90]:
mask2 = (comments['rating']==0) & (comments['sentiment']==False)

In [91]:
comments[mask2]

,user_id,recipe_id,date,rating,review,sentiment
12,468945,134728,2008-02-20,0,made my own buttermilk w/ vinegar and milk. u...,False
23,681408,225241,2008-03-14,0,this is a very good recipe. we also want to c...,False
67,2002097693,79222,2018-04-05,0,do you have cook the potatoes beforehand?,False
145,44642,190875,2006-10-19,0,"just posting, so i will know i tryed this. nev...",False
184,2001338551,195977,2017-04-04,0,have made this cake often. have added 1 1/2 c ...,False
...,...,...,...,...,...,...
1132282,176615,118119,2006-10-27,0,we've tried lots of peanut butter pasta recipe...,False
1132323,2405600,290157,2014-11-10,0,just added this mix to a homemade beef &amp; s...,False
1132325,2000650711,290157,2017-04-10,0,would this make a good hamburger patty seasoning?,False
1132361,102526,54493,2007-04-26,0,i am not going to rate this because i did have...,False


#### Remarques:

On constate qu'on "sauve" 25 935 commentaires notés à 0 qui pourtant contiennent un mot de vocabulaire positif.

In [92]:
comments.dropna(inplace= True)

In [93]:
comments

,user_id,recipe_id,date,rating,review,sentiment
0,38094,40893,2003-02-17,4,great with a salad. cooked on top of stove for...,True
1,1293707,40893,2011-12-21,5,"so simple, so delicious! great for chilly fall...",True
2,8937,44394,2002-12-01,4,this worked very well and is easy. i used not...,True
3,126440,85009,2010-02-27,5,i made the mexican topping and took it to bunk...,True
4,57222,85009,2011-10-01,5,"made the cheddar bacon topping, adding a sprin...",True
...,...,...,...,...,...,...
1132362,116593,72730,2003-12-09,0,another approach is to start making sauce with...,False
1132363,583662,386618,2009-09-29,5,these were so delicious! my husband and i tru...,True
1132364,157126,78003,2008-06-23,5,wow! sometimes i don't take the time to rate ...,True
1132365,53932,78003,2009-01-11,4,very good! i used regular port as well. the ...,True


Formatons nos données pour entrainer un modèle à prédire si un commentaire est positif ou négatif à partir du label sentiment:

In [94]:
y = comments['sentiment'][:30000]
x = comments['review'][:30000]

In [95]:
y

0        True
1        True
2        True
3        True
4        True
         ... 
30001    True
30002    True
30003    True
30004    True
30005    True
Name: sentiment, Length: 30000, dtype: bool

Partageons nos données en un ensemble d'apprentissage et un ensemble de test:

In [96]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

In [97]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline

model = make_pipeline(
        CountVectorizer(max_df= 0.5, ngram_range= (1, 2)),
        TfidfTransformer(),
        RandomForestClassifier()
)

model.fit(X_train, y_train)
print(f"Model score: {model.score(x, y):.2f}")

Model score: 0.99


In [98]:
preds =model.predict(X_test)

In [99]:
unique, counts = np.unique(preds, return_counts=True)

In [100]:
unique2, counts2 = np.unique(y_test, return_counts=True)

In [101]:
counts

array([   8, 4492])

In [102]:
counts2

array([ 240, 4260])

modèle rincé, classe déséquilibrées, faut score avec un F2 score